In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from matplotlib import pyplot as plt
plt.style.use('fivethirtyeight')

# Load Data

In [ ]:
data = pd.read_csv("../input/pizza-price-prediction/pizza_v1.csv")
data.head(10)

In [ ]:
data.info()

In [ ]:
# shape of data
data.shape

# Check missing value

In [ ]:
data.isnull().any()

# 

In [ ]:
# replace the of yes or no into 1,0

data = data.replace(["yes","no"],[1,0])
data.head()

# 

In [ ]:
##checing varities in company
data['company'].value_counts()

In [ ]:
# plot varities in company

plt.figure(figsize=(5,5))
data['company'].value_counts().plot(kind='pie',autopct='%.2f')
plt.title('Companies involved')
plt.ylabel('')
plt.show()

In [ ]:
# replace the of A, B, C, E or no into 0,1,2,3,4

data = data.replace(['A','B','C','D','E'],[0,1,2,3,4])
data['company']

# re-format price_rupiah column

In [ ]:
data['price_rupiah'].dtype
price = []
for item in data['price_rupiah']:
    price.append(int(item.replace('Rp','').replace(',','')))
data['price_rupiah'] = price
data.head()

# now just converting categorical data to numeric data

In [ ]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
data['topping']=labelencoder.fit_transform(data['topping'])
data['variant']=labelencoder.fit_transform(data['variant'])
data['size']=labelencoder.fit_transform(data['size'])
data.head()

# Correlation

In [ ]:
data.corr()

In [ ]:
corr = data.corr()
sns.set_context("notebook", font_scale=1.0, rc={"lines.linewidth": 2.5})
plt.figure(figsize=(10,5))
a = sns.heatmap(corr, annot=True, fmt='.1f', cmap='YlGnBu')
rotx = a.set_xticklabels(a.get_xticklabels(), rotation=90)
roty = a.set_yticklabels(a.get_yticklabels(), rotation=30)

In [ ]:
data.hist(figsize=(14,14))
plt.show()

# Slicing

In [ ]:
x = (data.loc[:, data.columns !='price_rupiah'])
y = (data.loc[:, data.columns =='price_rupiah'])

In [ ]:
from tensorflow.keras.utils import to_categorical
one_hot_labels = to_categorical(y)

In [ ]:
one_hot_labels

# Getting the statistical info

In [ ]:
x.describe()

# Split data into 70% training and 30% testing 

In [ ]:
import sklearn
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,one_hot_labels,test_size=.30,random_state=1)

# Normalize the train data

In [ ]:
mean = x_train.mean(axis=0)
x_train-= mean
std = x_train.std(axis=0)
x_train/= std
x_test-= mean
x_test/= std

# Create Model 

In [ ]:
from keras import models
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Dense
import tensorflow
network = models.Sequential()
network.add(Dense(32, activation='relu', input_shape=(x_train.shape[1],)))
network.add(Dense(8, activation='relu'))
network.add(Dense(1))

In [ ]:
network.compile(optimizer="rmsprop",loss="mse",metrics=["mae"])

In [ ]:
history=network.fit(x_train, y_train, epochs=10, verbose=1)

# evaluate(x_test,y_test)

In [ ]:
network.evaluate(x_test,y_test)

In [ ]:
result_1=network.predict(x_test)
result_1

In [ ]:
history_dict=history.history
history_dict.keys()

In [ ]:
loss = history_dict['loss']
mae = history_dict['mae']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss,"go",label='Training loss')
plt.plot(epochs, mae, 'r', label='mae')
plt.title('mae and  loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')

plt.legend()
plt.grid()
plt.show()